In [1]:
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians,sqrt
import numpy as np

In [2]:
data=pd.read_csv('C:/Users/hp/Downloads/case study -1/orders.csv')
data.head()

C:\Users\hp\.anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,16,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [3]:
data['promo_code'].fillna(0,inplace=True)
i=data.index[data.loc[:,'promo_code'] != 0]
data['promo_code'][i]=1

<ipython-input-3-2e0f102870f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['promo_code'][i]=1


In [4]:
data.promo_code_discount_percentage.fillna(0,inplace=True)
data.is_favorite.fillna('No',inplace=True)
data.item_count.fillna(data.item_count.median(),inplace=True)


In [5]:
data.drop(['driver_rating','deliverydistance','preparationtime','order_accepted_time','delivery_time','order_accepted_time', 'driver_accepted_time','ready_for_pickup_time', 'picked_up_time', 'delivered_time','delivery_date','created_at'],axis=1,inplace=True)

In [6]:
data

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,vendor_id,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,0,0.0,0.0,No,No,NaN,105,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,0,0.0,0.0,No,No,NaN,294,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,0,0.0,0.0,No,No,NaN,83,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,0,0.0,0.0,No,No,NaN,90,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,0,0.0,0.0,No,No,NaN,83,0,Work,BAL0RVT X 0 X 83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135298,415507.0,L6LFY6H,1.0,13.3,2,0,0.0,0.0,No,No,NaN,67,0,NaN,L6LFY6H X 0 X 67
135299,415508.0,2GMMVGI,3.0,9.5,1,0,0.0,0.0,No,Yes,5.0,79,0,Work,2GMMVGI X 0 X 79
135300,415510.0,LYTAAV0,4.0,18.2,2,0,0.0,0.0,No,No,NaN,28,2,Other,LYTAAV0 X 2 X 28
135301,415512.0,NEV9A5D,3.0,7.7,1,1,0.0,0.0,No,No,NaN,841,0,NaN,NEV9A5D X 0 X 841


In [7]:
customers=pd.read_csv('customers_info.csv')

In [8]:
new_data=pd.merge(customers,data,on='customer_id')

In [9]:
vendor=pd.read_csv('new_vendor.csv')

In [10]:
vendor

,vendor_id,ven_latitude,ven_longitude,vendor_category_en,delivery_charge,serving_distance,is_open,prepration_time,rank,primary_tags,vendor_rating,vendor_tag_name
0,4,-0.588596,0.754434,Restaurants,0.0,6.0,1.0,15,1,4,4.4,"Arabic,Breakfast,Burgers,Desserts,Free Deliver..."
1,13,-0.471654,0.744470,Restaurants,0.7,5.0,1.0,14,1,7,4.7,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa..."
2,20,-0.407527,0.643681,Restaurants,0.0,8.0,1.0,19,0,71,4.5,"Breakfast,Desserts,Free Delivery,Indian"
3,23,-0.585385,0.753811,Restaurants,0.0,5.0,1.0,16,1,46,4.5,"Burgers,Desserts,Fries,Salads"
4,28,0.480602,0.552850,Restaurants,0.7,15.0,1.0,10,1,32,4.4,Burgers
...,...,...,...,...,...,...,...,...,...,...,...,...
95,849,-1.588060,-0.066441,Restaurants,0.0,10.0,1.0,10,0,2396,4.1,"American,Breakfast,Burgers,Cafe,Desserts,Free ..."
96,855,2.145206,0.745025,Restaurants,0.0,8.0,1.0,10,0,2396,4.2,"American,Burgers,Desserts,Free Delivery,Fries,..."
97,856,0.251469,0.483632,Restaurants,0.0,7.0,1.0,10,1,2396,4.3,"American,Breakfast,Burgers,Cafe,Desserts,Free ..."
98,858,0.019817,0.587087,Restaurants,0.0,3.0,1.0,10,1,2396,4.2,"American,Breakfast,Burgers,Cafe,Desserts,Free ..."


In [10]:
final_data=pd.merge(new_data,vendor,on='vendor_id',how='left')

In [11]:
final_data


,customer_id,gender,location_type,longitude,latitude,akeed_order_id,item_count,grand_total,payment_mode,promo_code,...,ven_longitude,vendor_category_en,delivery_charge,serving_distance,is_open,prepration_time,rank,primary_tags,vendor_rating_y,vendor_tag_name
0,000THBA,male,Other,0.167026,-0.020250,222348.0,2.0,29.6,2,0,...,0.075989,Restaurants,0.7,15.0,1.0,16,1,134,4.1,"Breakfast,Burgers,Grills,Pizza,Sandwiches,Shaw..."
1,002510Y,female,Home,-78.597624,0.056822,126088.0,3.0,13.6,2,0,...,0.533995,Restaurants,0.7,15.0,1.0,15,1,297,4.3,"Biryani,Desserts,Indian,Rice,Thali,Vegetarian"
2,005ECL6,male,Work,0.158935,-0.315825,304576.0,5.0,23.4,1,0,...,0.081702,Restaurants,0.7,15.0,0.0,15,1,0,4.6,"American,Burgers,Desserts,Donuts,Fries,Pasta,S..."
3,005ECL6,male,Work,0.158935,-0.315825,336261.0,4.0,25.3,1,0,...,0.081702,Restaurants,0.7,15.0,0.0,15,1,0,4.6,"American,Burgers,Desserts,Donuts,Fries,Pasta,S..."
4,0075AM7,female,Other,-0.012308,0.080774,402450.0,1.0,25.8,1,0,...,0.072953,Restaurants,0.0,15.0,0.0,10,1,0,3.2,"Arabic,Pizzas,Sandwiches"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385532,ZZY3N0D,male,Home,-0.000951,-0.114377,292159.0,2.0,7.6,1,0,...,-0.062283,Restaurants,0.0,10.0,1.0,15,1,8,4.2,"Burgers,Free Delivery,Milkshakes,Mojitos"
385533,ZZY3N0D,male,Home,-0.000951,-0.114377,318689.0,1.0,7.9,1,0,...,0.067013,Restaurants,0.0,15.0,1.0,15,1,46,4.2,"American,Kids meal,Pasta,Pizzas,Salads"
385534,ZZY3N0D,male,Home,-0.000951,-0.114377,377037.0,1.0,8.4,1,0,...,0.039740,Restaurants,0.7,15.0,1.0,15,1,0,4.2,"Arabic,Kushari,Sandwiches"
385535,ZZY3N0D,male,Home,-0.000951,-0.114377,404527.0,1.0,7.1,1,0,...,0.052655,Restaurants,0.0,15.0,1.0,20,1,8,4.4,"American,Burgers,Desserts,Free Delivery,Fries,..."


In [12]:
ds=final_data.dropna()

In [13]:
ds

,customer_id,gender,location_type,longitude,latitude,akeed_order_id,item_count,grand_total,payment_mode,promo_code,...,ven_longitude,vendor_category_en,delivery_charge,serving_distance,is_open,prepration_time,rank,primary_tags,vendor_rating_y,vendor_tag_name
20,00GV4J4,male,Other,-78.559671,-0.264322,242001.0,1.0,18.7,1,0,...,0.551594,Restaurants,0.0,15.0,1.0,15,1,134,4.3,"American,Burgers,Kids meal,Pasta,Sandwiches,Sm..."
23,00GV4J4,male,Other,0.449445,-0.264730,242001.0,1.0,18.7,1,0,...,0.551594,Restaurants,0.0,15.0,1.0,15,1,134,4.3,"American,Burgers,Kids meal,Pasta,Sandwiches,Sm..."
26,00GV4J4,male,Home,0.079665,-1.575867,242001.0,1.0,18.7,1,0,...,0.551594,Restaurants,0.0,15.0,1.0,15,1,134,4.3,"American,Burgers,Kids meal,Pasta,Sandwiches,Sm..."
43,00HWUU3,male,Other,0.151387,-0.567823,266129.0,1.0,6.0,1,0,...,0.135124,Restaurants,0.7,15.0,1.0,15,1,1065,4.2,"Combos,Fries,Mojitos ,Pasta,Sandwiches"
49,00HWUU3,male,Home,0.103313,-1.117296,266129.0,1.0,6.0,1,0,...,0.135124,Restaurants,0.7,15.0,1.0,15,1,1065,4.2,"Combos,Fries,Mojitos ,Pasta,Sandwiches"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385529,ZZY3N0D,male,Home,-0.000951,-0.114377,251275.0,2.0,6.0,1,0,...,0.087738,Restaurants,0.7,15.0,1.0,10,1,701,4.3,"Sandwiches,Shawarma,Fresh Juices,Mojitos,Milks..."
385530,ZZY3N0D,male,Home,-0.000951,-0.114377,265699.0,1.0,8.2,1,0,...,0.058017,Sweets & Bakes,0.0,15.0,1.0,13,1,336,4.5,"Cakes,Crepes,Desserts,Donuts,Fresh Juices,Ice ..."
385531,ZZY3N0D,male,Home,-0.000951,-0.114377,285179.0,2.0,4.1,1,0,...,0.091491,Restaurants,0.0,15.0,1.0,10,1,346,4.2,"Breakfast,Free Delivery,Fresh Juices,Fries,Kar..."
385532,ZZY3N0D,male,Home,-0.000951,-0.114377,292159.0,2.0,7.6,1,0,...,-0.062283,Restaurants,0.0,10.0,1.0,15,1,8,4.2,"Burgers,Free Delivery,Milkshakes,Mojitos"


In [14]:
ds.drop_duplicates(inplace=True)

<ipython-input-14-3800e7188387>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds.drop_duplicates(inplace=True)


In [15]:
ds.to_csv('data.csv')